# __Analysing Tweets to Verify the Presence of Toxic Speech With ToLD-Br__

In [1]:
import pandas as pd
from IPython.display import clear_output
from simpletransformers.classification import ClassificationModel

/home/lucas/anaconda3/envs/twitter-cpi/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# data
tweets_df = pd.read_parquet('data/networks/complete_group_interacs.parquet')
tweets_df.shape

(1272586, 5)

In [3]:
tweets_df.sample(10)

,source,target,factions,interac_type,tweet_content
362928,FeSeu,EduGiraoOficial,pro gov to neutral,mention,Renan afirma tópicos que foram negados por tod...
1197952,pensadorpensa,g1,pro gov to neutral,reply,@g1 A #CPIdoCirco em ação.
570695,Esbaile,EduGiraoOficial,pro gov to neutral,quoted_rt,"É o óbvio que, ao ser ouvido dá NOJO E REVOLTA..."
292920,Cristin07229113,SenadoFederal,pro gov to neutral,reply,@SenadoFederal O que dificutou foram as mentir...
795607,Neneto,MarcosRogerio,neutral to neutral,mention,@MarcosRogerio porque o senhor está defendendo...
787295,tatyanecibele,MarcosRogerio,neutral to neutral,mention,Resumindo @MarcosRogerio nao agrega nada repet...
918753,draChristina22,TSEjusbr,pro gov to neutral,reply,@TSEjusbr @policiafederal Queremos VOTO IMPRES...
217395,MidiaNINJA,randolfeap,neutral to neutral,mention,Acompanhou a #CPIdoCOVID hoje? Quer saber quai...
622909,emmaiafilho,camaradeputados,pro gov to neutral,mention,Alguma representação ou bancada feminina da @c...
186834,Zambele_se,STF_oficial,against gov to neutral,mention,Esse verme não tem respeito nem pela pai dos s...


In [4]:
def x_perc_of(v1, v2):
    return round((v1*v2)/100)

In [5]:
tweets_df['factions'].value_counts()

pro gov to pro gov            420024
against gov to against gov    233985
against gov to neutral        210992
pro gov to neutral            150756
neutral to neutral            128118
pro gov to against gov         45385
against gov to pro gov         36404
neutral to against gov         36088
neutral to pro gov             10834
Name: factions, dtype: int64

In [ ]:
total_size = tweets_df[tweets_df['factions']=='pro gov to pro gov'].shape[0]
pro_gov2pro_gov_sample = tweets_df[tweets_df['factions']=='pro gov to pro gov'].sample(x_perc_of(20, total_size))
pro_gov2pro_gov_sample.shape

In [ ]:
total_size = tweets_df[tweets_df['factions']=='against gov to against gov'].shape[0]
ant_gov2ant_gov_sample = tweets_df[tweets_df['factions']=='against gov to against gov'].sample(x_perc_of(20, total_size))
ant_gov2ant_gov_sample.shape

In [ ]:
total_size = tweets_df[tweets_df['factions']=='against gov to neutral'].shape[0]
ant_gov2neutral_sample = tweets_df[tweets_df['factions']=='against gov to neutral'].sample(x_perc_of(20, total_size))
ant_gov2neutral_sample.shape

In [ ]:
total_size = tweets_df[tweets_df['factions']=='pro gov to neutral'].shape[0]
pro_gov2neutral_sample = tweets_df[tweets_df['factions']=='pro gov to neutral'].sample(x_perc_of(20, total_size))
pro_gov2neutral_sample.shape

In [ ]:
total_size = tweets_df[tweets_df['factions']=='neutral to neutral'].shape[0]
neutral2neutral_sample = tweets_df[tweets_df['factions']=='neutral to neutral'].sample(x_perc_of(20, total_size))
neutral2neutral_sample.shape

In [ ]:
sample_list = [pro_gov2pro_gov_sample, ant_gov2ant_gov_sample, ant_gov2neutral_sample, pro_gov2neutral_sample, neutral2neutral_sample]
prediction_sample_list = []

In [8]:
model = ClassificationModel("distilbert", "toxic_bert_model", use_cuda=False)

In [ ]:
count = 0

for df in sample_list:
    sample_predicts = []
    for tweet in df['tweet_content'].tolist():
        clear_output(wait=False)
        print(f'Sample number {count} in process...')
        prediction, output = model.predict([tweet])
        sample_predicts.append(prediction[0])
    prediction_sample_list.append(sample_predicts)
    count+=1

In [ ]:
# pred_list = []
# count = 0

# for tweet in tweets_df['tweet_content'].tolist():
#     clear_output(wait=False)
#     print('Tweet number: ', count)
#     prediction, output = model.predict([tweet])
#     pred_list.append(prediction[0])
#     count+=1
# #print("Predictions:", predictions)
# #print("Outputs:", outputs)

In [18]:
%%time
predictions, outputs = model.predict(tweets_df[tweets_df['factions']=='pro gov to pro gov'].sample(2000)['tweet_content'].tolist())

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

CPU times: user 19min 55s, sys: 17 s, total: 20min 12s
Wall time: 5min 6s


In [17]:
pd.Series(predictions).value_counts()

0    2322
1     678
dtype: int64

In [40]:
n = 3
n_samples = 2000
sample_dict = {}
predict_dict = {}
slices = ['pro gov to pro gov', 'against gov to against gov', 'against gov to neutral',
          'pro gov to neutral', 'neutral to neutral', 'pro gov to against gov',
          'against gov to pro gov']


In [41]:
%%time

for slice in slices:
    sample_dict[slice] = []
    predict_dict[slice] = []
    print(f'Starting prediction slice: {slice}')
    for x in range(n):
        print(f'Starting prediction number {x}...')
        samples = tweets_df[tweets_df['factions']==slice].sample(n_samples)
        sample_dict[slice].append(samples)
        predictions, outputs = model.predict(samples['tweet_content'].tolist())
        predict_dict[slice].append(predictions)

Starting prediction slice: pro gov to pro gov
Starting prediction number 0...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

Starting prediction number 1...


  0%|          | 0/2000 [00:00<?, ?it/s]

...

  0%|          | 0/250 [00:00<?, ?it/s]

Starting prediction number 2...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

CPU times: user 7h 17min 8s, sys: 6min 25s, total: 7h 23min 34s
Wall time: 1h 52min 27s


In [44]:
pd.Series(predict_dict['pro gov to pro gov'][1]).value_counts()

0    1539
1     461
dtype: int64

In [46]:
pd.Series(predict_dict['against gov to against gov'][0]).value_counts()

0    1666
1     334
dtype: int64

In [49]:
pd.Series(predict_dict['against gov to neutral'][2]).value_counts()

0    1547
1     453
dtype: int64

In [52]:
pd.Series(predict_dict['pro gov to neutral'][2]).value_counts()

0    1241
1     759
dtype: int64

In [55]:
pd.Series(predict_dict['pro gov to against gov'][1]).value_counts()

0    1238
1     762
dtype: int64

In [56]:
pd.Series(predict_dict['against gov to pro gov'][1]).value_counts()

0    1289
1     711
dtype: int64

In [59]:
pd.Series(predict_dict['neutral to neutral'][2]).value_counts()

0    1615
1     385
dtype: int64

In [63]:
tweets_df[tweets_df['target']=='OmarAzizSenador'].sample()

,source,target,factions,interac_type,tweet_content
481912,rsaraivaaa,OmarAzizSenador,neutral to neutral,mention,@OmarAzizSenador dando uma mijada de leve nas ...


In [64]:
tweets_df[tweets_df['target']=='randolfeap'].sample()

,source,target,factions,interac_type,tweet_content
725017,dulcinealisboa,randolfeap,neutral to neutral,reply,@randolfeap @AnaPaul11890999 Este tal domighet...


In [65]:
tweets_df[tweets_df['target']=='renancalheiros'].sample()

,source,target,factions,interac_type,tweet_content
272409,Eduhst,renancalheiros,neutral to neutral,mention,"Vixi, a Mayra citou o @RodrigoMenegat como o ""..."


In [68]:
tweets_df[tweets_df['target']=='taoquei1'].sample()

,source,target,factions,interac_type,tweet_content
230579,LidiaCa28545681,taoquei1,pro gov to pro gov,quoted_rt,#RenanVagabundo gastando dinheiro público só p...


In [69]:
tweets_df[tweets_df['target']=='CarlaZambelli38'].sample()

,source,target,factions,interac_type,tweet_content
411051,AparecidaSaler3,CarlaZambelli38,pro gov to pro gov,quoted_rt,"Com certeza, s dúvida alguma!!#LulaLadrao http..."


In [70]:
tweets_df[tweets_df['target']=='MarcosRogerio'].sample()

,source,target,factions,interac_type,tweet_content
483565,elcivanni,MarcosRogerio,pro gov to neutral,reply,@MarcosRogerio @FlavioBolsonaro Interrompa! Pe...


In [71]:
tweets_df[tweets_df['target']=='kimpaim'].sample()

,source,target,factions,interac_type,tweet_content
964417,DireiBrasileira,kimpaim,pro gov to pro gov,reply,@kimpaim @antonio69183994 Caso contrário é #Ba...


In [72]:
tweets_df[tweets_df['target']=='SenadoFederal'].sample()

,source,target,factions,interac_type,tweet_content
802215,mad13257,SenadoFederal,pro gov to neutral,reply,@SenadoFederal -\n [...impulsionada pel...


In [73]:
tweets_df[tweets_df['target']=='Rconstantino'].sample()

,source,target,factions,interac_type,tweet_content
556367,ORLANDOSALUSTI2,Rconstantino,pro gov to pro gov,reply,@Rconstantino Boa noite!\n😴🙏🏽🇧🇷🙏🏽\n#AGloboNãoP...


In [74]:
tweets_df[tweets_df['target']=='BolsonaroSP'].sample()

,source,target,factions,interac_type,tweet_content
560194,SheylaPereirad3,BolsonaroSP,pro gov to pro gov,quoted_rt,Bolsonaro está certo.\n#AGloboNaoPresta https:...


In [78]:
tweets_df[tweets_df['target']=='senadorhumberto'].sample()

,source,target,factions,interac_type,tweet_content
1245114,katialeticia112,senadorhumberto,against gov to against gov,quoted_rt,Resta saber se teremos punição severa pra TODO...


In [79]:
tweets_df[tweets_df['target']=='folha'].sample()

,source,target,factions,interac_type,tweet_content
367483,BiaRMesquita,folha,neutral to neutral,quoted_rt,O Brasil vai matar mais gente por Covid-19 até...


In [80]:
tweets_df[tweets_df['target']=='UOLNoticias'].sample()

,source,target,factions,interac_type,tweet_content
41234,LucasPazoline3,UOLNoticias,pro gov to neutral,reply,@UOLNoticias ATENÇÃO: Essa é a tag que precisa...


In [86]:
tweets_df[tweets_df['target']=='STF_oficial'].sample()

,source,target,factions,interac_type,tweet_content
440393,_karrlinhos_,STF_oficial,pro gov to neutral,mention,CPI _URGENTE\n\nGov @wilsonlimaAM do @pscnacio...


In [87]:
tweets_df[tweets_df['target']=='Sen_Alessandro'].sample()

,source,target,factions,interac_type,tweet_content
788017,KeziaFer,Sen_Alessandro,neutral to neutral,mention,@Sen_Alessandro mais uma vez craque do dia !! ...


In [88]:
tweets_df[tweets_df['target']=='Metropoles'].sample()

,source,target,factions,interac_type,tweet_content
1162334,MATHFREIREB_TW,Metropoles,neutral to neutral,reply,@Metropoles @mqueiroga2 A gota d'água – um Min...


In [89]:
tweets_df[tweets_df['target']=='MidiaNINJA'].sample()

,source,target,factions,interac_type,tweet_content
435956,esterd8,MidiaNINJA,neutral to neutral,reply,@MidiaNINJA Outro absurdo que ninguém fala: os...


In [90]:
tweets_df[tweets_df['target']=='Estadao'].sample()

,source,target,factions,interac_type,tweet_content
1102728,ItajahyCarol,Estadao,against gov to neutral,reply,@Estadao IMPEACHMENT URGENTE!!! Cadeia para B...


In [94]:
tweets_df[tweets_df['target']=='MarceloFreixo'].sample()

,source,target,factions,interac_type,tweet_content
772507,jessymr,MarceloFreixo,against gov to neutral,mention,"Deputados, rogo suas assinaturas na emenda Nº:..."


In [95]:
tweets_df[tweets_df['target']=='GuilhermeBoulos'].sample()

,source,target,factions,interac_type,tweet_content
577521,martinsbruno021,GuilhermeBoulos,against gov to against gov,reply,@GuilhermeBoulos O Risco é grande já que a tro...


In [98]:
tweets_df[tweets_df['target']=='o_antagonista'].sample()

,source,target,factions,interac_type,tweet_content
1065047,ABracet,o_antagonista,against gov to against gov,reply,@o_antagonista Orando aqui: #Forabolsonaro. Amém.


In [100]:
tweets_df[tweets_df['target']=='ContaratoSenado'].sample()

,source,target,factions,interac_type,tweet_content
790538,renatalommez,ContaratoSenado,against gov to against gov,mention,Alô @slpng_giants_pt o canal de propagação de ...


In [105]:
tweets_df[tweets_df['target']=='cirogomes'].sample()

,source,target,factions,interac_type,tweet_content
565305,Amanda22706722,cirogomes,against gov to against gov,reply,@cirogomes E o Arthur Lira não faz nada #ForaB...


In [106]:
tweets_df[tweets_df['target']=='juliette'].sample()

,source,target,factions,interac_type,tweet_content
535990,tuittadry,juliette,against gov to against gov,reply,@juliette É ISSO!! #ForaBolsonaro https://t.co...


In [107]:
tweets_df[tweets_df['target']=='brunogagliasso'].sample()

,source,target,factions,interac_type,tweet_content
562194,sol_maryh,brunogagliasso,against gov to against gov,reply,@brunogagliasso Você e @gioewbank são maravilh...


In [122]:
tweets_df[tweets_df['target']=='MBLivre'].sample()

,source,target,factions,interac_type,tweet_content
433272,cirosaopaulino,MBLivre,against gov to against gov,reply,"@MBLivre @YouTube Blz, faliu o MBL, que apoiou..."


In [127]:
tweets_df[tweets_df['target']=='jdoriajr'].sample()

,source,target,factions,interac_type,tweet_content
900041,laffurr2,jdoriajr,against gov to against gov,reply,@jdoriajr Quando a violência parte do governo ...


In [128]:
tweets_df[tweets_df['target']=='veramagalhaes'].sample()

,source,target,factions,interac_type,tweet_content
222638,JLSP40,veramagalhaes,neutral to neutral,reply,@veramagalhaes @veramagalhaes pq quando os âni...


In [129]:
tweets_df[tweets_df['target']=='RubensNunesMBL'].sample()

,source,target,factions,interac_type,tweet_content
1071091,CalefiLeandro,RubensNunesMBL,against gov to against gov,quoted_rt,12! de Setembro! \n#12SetEuVou https://t.co/FI...


In [133]:
tweets_df[tweets_df['target']=='KimKataguiri'].sample()

,source,target,factions,interac_type,tweet_content
1121477,colheitadeira,KimKataguiri,against gov to against gov,reply,@KimKataguiri 12 de setembro é #ForaBolsonaro


In [137]:
tweets_df[tweets_df['factions']=='against gov to against gov'].sample(20)

,source,target,factions,interac_type,tweet_content
1154862,DenizeBocchi6,mmarescast,against gov to against gov,reply,@mmarescast Pirou de vez. #2OutForaBolsonaro #...
742105,layane_sarmento,jefnogueira,against gov to against gov,quoted_rt,Comida no prato\nVacina no braço\nFora Bolsona...
477804,noemi_nk,o_antagonista,against gov to against gov,reply,@o_antagonista Vacina pra mim cloroquina e inv...
740432,leilasoraya,ptbrasil,against gov to against gov,quoted_rt,Congonhas é #3JForaBolsonaro #3JPovoNasRuas ht...
722021,ThiagoRotta,bslvra,against gov to against gov,quoted_rt,"Pego no flagra, ou deveria dizer pego com as c..."
1113353,Gilengajar,GilDoVigor,against gov to against gov,reply,@GilDoVigor Orgulho #ForaBolsonaro
332945,bem_nice,joaoluizpedrosa,against gov to against gov,reply,@joaoluizpedrosa Bom te ver militando! #29MFor...
600200,CarmemA68663100,Daslei,against gov to against gov,quoted_rt,Bolsonaro vai cair!\n#impeachmentDoBolsonaro \...
625776,vbertolino,GuilhermeBoulos,against gov to against gov,reply,@GuilhermeBoulos Cadeia nele.\n#BolsonaroCorru...
1233050,epicurodepedra,erikakokay,against gov to against gov,quoted_rt,Brasília no #ForaBolsonaro https://t.co/AutEPh...


In [140]:
tweets_df[tweets_df['factions']=='pro gov to neutral']['target'].value_counts()

OmarAzizSenador    21213
SenadoFederal      19172
MarcosRogerio       8611
renancalheiros      8171
SigaGazetaBR        7692
                   ...  
joseleonidas           1
pinkbloc_              1
seujosias1             1
true_news_now          1
marianasendoela        1
Name: target, Length: 2236, dtype: int64

In [141]:
tweets_df[tweets_df['factions']=='against gov to neutral']['target'].value_counts()

AAMachadoRP        12301
randolfeap         11253
UOLNoticias         9657
OmarAzizSenador     8223
MidiaNINJA          8044
                   ...  
ootimistajornal        1
LilianaRochaF          1
cruzeiro_mauro         1
printandonews          1
CarlosHortmann         1
Name: target, Length: 5116, dtype: int64

In [144]:
tweets_df[tweets_df['factions']=='pro gov to against gov']['target'].value_counts()

senadorhumberto    6176
revistaoeste       5839
o_antagonista      4790
jandira_feghali    1996
joaoamoedonovo     1385
                   ... 
JoaoAPedroso          1
thiago10galo          1
cade_lili             1
MarxJuca              1
fabiolabaani          1
Name: target, Length: 5095, dtype: int64